# Proje: Extract Text from PDF

Python kullanarak PDF dosyalarından nasıl metin çıkarabileceğinizi anlatacağız. PDF'den metin çıkarmak kolay bir iş değildir, burada yapılacak çok şey vardır. Ancak biraz yardım için, pip komutu kullanılarak kolayca kurulabilen pdf2image olarak bilinen bir Python paketi kullanacağız.

In [10]:
 #pip install pdf2image

PDF dosyasından metin çıkarırken karşılaştığımız en büyük zorluk, PDF dosyalarının farklı dosya formatlarında gelmesidir. Bu nedenle öncelikle bir PDF dosyasının her formatını istediğimiz formata dönüştürebileceğimiz bir fonksiyon hazırlamamız gerekir.

In [11]:
!apt-get install poppler-utils # Install poppler-utils, which contains pdfinfo
!pip install pdf2image # Make sure pdf2image is installed
!apt-get install tesseract-ocr
!pip install pytesseract
!sudo apt-get install tesseract-ocr-tur

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-tur is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [12]:
#pip install pytesseract

In [13]:
import pdf2image
import os, sys
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

Şimdi, pdf çıkarma fonksiyonunda klasördeki belgelerinizi saymak için daha sonra kullanılacak olan belgelerinizin yolunu ve sayacı başlatmamız gerekiyor

In [14]:
PATH = '/content/'

#initialize the counter that you will use later in your pdf extraction function
i = 1

Şimdi, pdf dosyalarımızdan bazı gereksiz dosyaları silmemiz gerekiyor, bunun için yeni bir fonksiyon oluşturacağız

In [15]:
def delete_ppms():
  for file in os.listdir(PATH):
    if '.ppm' in file or '.DS_Store' in file:
      try:
          os.remove(PATH + file)
      except FileNotFoundError:
          pass

Şimdi pdf dosyalarını türlerine göre sıralamamız gerekiyor. Bunu pdf dosyaları için bir ve Docx dosyaları için bir liste oluşturarak başlatacağım çünkü bu iki tür en çok kullanılan pdf dosya türleridir

In [16]:
pdf_files = []
docx_files = []

# append document names into the lists by their extension type
for f in os.listdir(PATH):
  full_name = os.path.join(PATH, f)
  if os.path.isfile(full_name):
    name = os.path.basename(f)
    filename, ext = os.path.splitext(name)
    if ext == '.pdf':
      pdf_files.append(name)
    elif ext == ('.docx'):
      docx_files.append(name)

Artık sonunda PDF dosyalarından metin çıkarabiliriz. İşte pdf_extract fonksiyonu. İlk olarak, metnin çıkarıldığı her dosyanın adını yazdırır. Belgenin boyutuna bağlı olarak, metnin çıkarılması biraz zaman alabilir

In [17]:
import os
import pdf2image
from PIL import Image
import pytesseract

# Specify the path to the tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

def pdf_extract(file, i):
  print("extracting from file:", file)
  delete_ppms()  # Assuming delete_ppms is defined elsewhere
  # Add poppler_path argument to convert_from_path
  images = pdf2image.convert_from_path(PATH + file, output_folder=PATH, poppler_path="/usr/bin")  # Assuming PATH is defined elsewhere
  j = 0
  for file in sorted (os.listdir(PATH)):
      if '.ppm' in file and 'image' not in file:
        os.rename(PATH + file, PATH + 'image' + str(i) + '-' + str(j) + '.ppm')
        j += 1
  j = 0
  f = open(PATH +'result{}.txt'.format(i), 'w')
  files = [f for f in os.listdir(PATH) if '.ppm' in f]

  for file in sorted(files, key=lambda x: int(x[x.index('-') + 1: x.index('.')])):
      # Specify Turkish language using 'tur'
      temp = pytesseract.image_to_string(Image.open(PATH + file), lang='tur')
      f.write(temp)
  f.close()

Şimdi, Python kullanarak tüm PDF dosyalarından veri çıkarmak için fonksiyonumuzu kullanabiliriz

In [18]:
for i in range(len(pdf_files)):
  pdf_file = pdf_files[i]
  pdf_extract(pdf_file, i)

extracting from file: 35144133048_KayitYenilemeDokum_ders.pdf


Şimdi fonksiyonu çalıştırdıktan sonra dizine giderseniz result1.txt adında PDF dosyasından çıkarılan tüm metinleri içeren bir metin dosyası göreceksiniz.